In [1]:
import os
import config
import langchain.llms
import google.generativeai as genai


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_key = os.environ.get('API_KEY')

## creating an SQL database object

In [18]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "root123"
db_host = "localhost"
db_name = "atliq_tshirts"

try:
    # Attempt to create a database connection
    db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
    print("DATABASE CONNECTED SUCCESSFULLY")
except Exception as e:
    # Print failure message and the specific error
    print("DATABASE CONNECTION FAILED")
    print(f"Error: {str(e)}")
    exit(1)

DATABASE CONNECTED SUCCESSFULLY


### creating an SQL chain

In [40]:
from langchain_experimental.sql import SQLDatabaseChain